In [12]:
import sys

sys.path.append("c:\\Users\\robin\\Documents\\HyperBrain")
sys.path.append("c:\\Users\\robin\\Documents\\HyperBrain\\source")

In [13]:
from source.datasets.brain_dataset import BrainDataset
from torch.utils.data import DataLoader
from source.lightning.setup_model import setup_lightning_loftr, LitLoFTR
import lightning as L
from torchvision.transforms import v2
from source.datasets.brain_dataset import collate_fn
from lightning.pytorch.callbacks import DeviceStatsMonitor
from source.loftr.backbone import ResNetFPN_16_4, ResNetFPN_8_2
from source.loftr.positional_encoding import PositionalEncoding
from source.loftr.transformer import LocalFeatureTransformer
from source.loftr.coarse_matching import CoarseMatching
from source.loftr.fine_matching import FineMatching
from source.loftr.fine_preprocess import FinePreprocess
from source.loftr.loss import *

In [14]:
crop_size = 640
affine_transformation_range = 0.25
perspective_transformation_range = 0.0001
patch_size = 16
max_translation_shift = 50
fine_height_width = (crop_size//patch_size)*4
coarse_height_width = crop_size//patch_size
images_directory = "../../data/cyto_downscaled_3344_3904/"
use_train_data = True

dataset_train = BrainDataset(
    images_directory=images_directory,
    train=use_train_data,
    affine_transformation_range=affine_transformation_range,
    perspective_transformation_range=perspective_transformation_range,
    crop_size=crop_size,
    patch_size=patch_size,
    max_translation_shift=max_translation_shift,
    fine_height_width=fine_height_width,
    transform=v2.Compose([v2.Normalize(mean=[0.594], std=[0.204])]),
)

dataloader_train = DataLoader(
    dataset=dataset_train,
    batch_size=1,
    shuffle=True,
    pin_memory=True,
    drop_last=False,
    collate_fn=collate_fn,
    # num_workers=11
)

In [5]:
block_dimensions = [64, 96, 128, 192]
temperature = 0.2
use_coarse_context = False
fine_loss = "l2_std"


backbone = ResNetFPN_8_2(block_dimensions=block_dimensions)
fine_feature_size = block_dimensions[0]

coarse_feature_size = block_dimensions[-1]

positional_encoding = PositionalEncoding(coarse_feature_size)

coarse_loftr = LocalFeatureTransformer(
feature_dimension=coarse_feature_size,
number_of_heads=8,
layer_names=["self", "cross"] * 4,
)

coarse_matcher = CoarseMatching(temperature=temperature, confidence_threshold=0.2)

fine_preprocess = FinePreprocess(
coarse_feature_size=coarse_feature_size,
fine_feature_size=fine_feature_size,
window_size=5,
use_coarse_context=use_coarse_context,
)

fine_loftr = LocalFeatureTransformer(
feature_dimension=fine_feature_size,
number_of_heads=8,
layer_names=["self", "cross"],
)

use_l2_with_standard_deviation = True if fine_loss == "l2_std" else False
fine_matching = FineMatching(
return_standard_deviation=use_l2_with_standard_deviation,
)

In [6]:
loftr = LitLoFTR.load_from_checkpoint("lightning_logs/version_1/checkpoints/epoch=215-step=216.ckpt", 
                                      backbone=backbone, positional_encoding=positional_encoding, 
                                      coarse_loftr=coarse_loftr, coarse_matcher=coarse_matcher, 
                                      fine_preprocess=fine_preprocess, fine_loftr=fine_loftr, 
                                      fine_matching=fine_matching, fine_loss=fine_loss, coarse_loss="official_focal")

In [11]:
loftr(torch.Tensor([1,]))

NotImplementedError: Module [LitLoFTR] is missing the required "forward" function